# Nome: Jhonnatha Luiz de Souza Carvalho.
# Nome : Paulo Ricardo Lima.

# Parte I

## 1. Instalando Postgres


In [5]:
!sudo apt install -y postgresql


Lendo listas de pacotes... Pronto
Construindo árvore de dependências... Pronto
Lendo informação de estado... Pronto        
Os pacotes adicionais seguintes serão instalados:
  postgresql-17 postgresql-client-17 postgresql-client-common
  postgresql-common
Pacotes sugeridos:
  postgresql-doc postgresql-doc-17
Os NOVOS pacotes a seguir serão instalados:
  postgresql postgresql-17 postgresql-client-17 postgresql-client-common
  postgresql-common
0 pacotes atualizados, 5 pacotes novos instalados, 0 a serem removidos e 41 não atualizados.
É preciso baixar 0 B/19,4 MB de arquivos.
Depois desta operação, 69,5 MB adicionais de espaço em disco serão usados.
Pré-configurando pacotes ...

78A seleccionar pacote anteriormente não seleccionado postgresql-client-common.
(Lendo banco de dados ... 216012 ficheiros e diretórios atualmente instalados.)
A preparar para desempacotar .../postgresql-client-common_264.pgdg24.04+1_all.deb ...
7Progress: [  0%] [.............................................

In [4]:
!pg_config --version

PostgreSQL 17.0 (Ubuntu 17.0-1.pgdg24.04+1)


In [1]:
!git clone https://github.com/foliveirafilho/tpch-pgsql.git

Cloning into 'tpch-pgsql'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 164 (delta 0), reused 0 (delta 0), pack-reused 160 (from 1)
Receiving objects: 100% (164/164), 774.38 KiB | 2.05 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [2]:
!sudo mv Tp3.ipynb tpch-pgsql/

# Reabra o arquivo jupyter que foi movido para pasta tpch-psql

In [1]:
%pip install -r requirements.txt --break-system-packages


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## 2. Geração de BD em testes:
- contém passos que foram seguidos do link do github da especificalção.

In [2]:
# Instalar PostgreSQL e contribuições
!sudo apt-get install -y postgresql postgresql-contrib

# Criar o usuário e o banco de dados
!sudo -u postgres createuser tpch
!sudo -u postgres createdb tpchdb


Lendo listas de pacotes... Pronto
Construindo árvore de dependências... Pronto
Lendo informação de estado... Pronto        
postgresql já é a versão mais recente (17+264.pgdg24.04+1).
postgresql-contrib já é a versão mais recente (17+264.pgdg24.04+1).
0 pacotes atualizados, 0 pacotes novos instalados, 0 a serem removidos e 41 não atualizados.
createuser: erro: criação de nova role falhou: ERRO:  a função de banco de dados (role) "tpch" já existe
createdb: erro: criação do banco de dados falhou: ERRO:  o banco de dados "tpchdb" já existe


## Como houve problemas mudamos a senha do usuário 'postgres' para o uso mais simples.

In [ ]:
# Configurar o usuário 'tpch' com senha e permissões para o banco de dados 'tpchdb'
!sudo -u postgres psql -c "ALTER USER tpch WITH ENCRYPTED PASSWORD 'test123';"
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE tpchdb TO tpch;"

!sudo -u postgres psql -c "ALTER USER postgres WITH ENCRYPTED PASSWORD 'test123';"


ALTER ROLE
GRANT
ALTER ROLE


In [4]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\l
EOF


                                                       Lista de bancos de dados
   Nome    |   Dono   | Codificação | Provedor de localidade |  Ordenação  |    Ctype    | Locale | Regras ICU | Privilégios de acesso 
-----------+----------+-------------+------------------------+-------------+-------------+--------+------------+-----------------------
 postgres  | postgres | UTF8        | libc                   | pt_BR.UTF-8 | pt_BR.UTF-8 |        |            | 
 template0 | postgres | UTF8        | libc                   | pt_BR.UTF-8 | pt_BR.UTF-8 |        |            | =c/postgres          +
           |          |             |                        |             |             |        |            | postgres=CTc/postgres
 template1 | postgres | UTF8        | libc                   | pt_BR.UTF-8 | pt_BR.UTF-8 |        |            | =c/postgres          +
           |          |             |                        |             |             |        |            | postgres=CTc/p

In [5]:
%%bash
wget -q https://github.com/electrum/tpch-dbgen/archive/32f1c1b92d1664dba542e927d23d86ffa57aa253.zip -O tpch-dbgen.zip
unzip -q tpch-dbgen.zip && mv tpch-dbgen-32f1c1b92d1664dba542e927d23d86ffa57aa253 tpch-dbgen && rm tpch-dbgen.zip

In [6]:
!python3 tpch_pgsql.py prepare -H localhost -p 5432 -U postgres -W 'test123' -d tpchdb -i ./data -q ./query_root -g ./tpch-dbgen 

gcc -g -DDBNAME=\"dss\" -DLINUX -DORACLE -DTPCH -DRNG_TEST -D_FILE_OFFSET_BITS=64   -c -o build.o build.c
In file included from build.c:45:
build.c: In function ‘mk_time’:
dss.h:487:14: warning: ‘%02ld’ directive writing between 2 and 17 bytes into a region of size 11 []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-overflow=-Wformat-overflow=]8;;]
  487 | sprintf(tgt, "19%02ld-%02ld-%02ld", yr, mn, dy)
      |              ^~~~~~~~~~~~~~~~~~~~~
build.c:420:9: note: in expansion of macro ‘PR_DATE’
  420 |         PR_DATE(t->alpha, y, m,
      |         ^~~~~~~
dss.h:487:17: note: format string is defined here
  487 | sprintf(tgt, "19%02ld-%02ld-%02ld", yr, mn, dy)
      |                 ^~~~~
dss.h:487:14: note: directive argument in the range [-9223372036854775, 9223372036854775]
  487 | sprintf(tgt, "19%02ld-%02ld-%02ld", yr, mn, dy)
      |              ^~~~~~~~~~~~~~~~~~~~~
build.c:420:9: note: in expansion of macro ‘PR_DATE’
  420 |         PR_DATE(t

In [7]:
!python3 tpch_pgsql.py load -H localhost -p 5432 -U postgres -W 'test123' -d tpchdb -i ./data -q ./query_root -g ./tpch-dbgen 

dropped existing tables
cleaned database tpchdb
done creating schemas
done loading data to tables
done creating indexes and foreign keys
=========================== Load ===========================
create_schema: : 0:00:00.021424
load_data: 0:00:20.943801
index_tables: 0:00:13.942482
======================= End Results ========================


## 3. Execução de Consultas:



In [8]:
!python3 tpch_pgsql.py query -H localhost -p 5432 -U postgres -W 'test123' -d tpchdb -i ./data -q ./query_root -g ./tpch-dbgen

Power tests started ...
Power tests finished.
Throughput tests started ...
Throughput tests in stream #1 started ...
Throughput tests in stream #2 started ...
Throughput tests finished.
done performance tests
Power@Size = 8262.045461478283
Throughput@Size = 8234.331182824766
QphH@Size = 8248.1766819926
========================= Metrics ==========================
power_size: 8262.045461478283
throughput_size: 8234.331182824766
qphh_size: 8248.1766819926
======================= End Results ========================


## 4. Informações sobre o Hardware

### Tipo de Processador
    Nome do modelo:             AMD Ryzen 7 3700X 8-Core Processor
    Família da CPU:           23
    Modelo:                   113
    Thread(s) per núcleo:     2
    Núcleo(s) por soquete:    8


### Quantidade de Memória RAM
    32 GB.


### Tamanho do Disco
    1 TB.


### Informações sobre as Caches
    cache de L1d:                         256 KiB (8 instances)
    cache de L1i:                         256 KiB (8 instances)
    cache de L2:                          4 MiB (8 instances)
    cache de L3:                          32 MiB (2 instances)


### Versão do Kernel
    Linux kurokym-B450M-GAMING 6.8.0-48-generic #48-Ubuntu SMP PREEMPT_DYNAMIC Fri Sep 27 14:04:52 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux
    No LSB modules are available.
    Distributor ID:	Ubuntu
    Description:	Ubuntu 24.04.1 LTS
    Release:	24.04
    Codename:	noble


In [9]:
!lscpu

Arquitetura:                  x86_64
  Modo(s) operacional da CPU: 32-bit, 64-bit
  Address sizes:              43 bits physical, 48 bits virtual
  Ordem dos bytes:            Little Endian
CPU(s):                       16
  Lista de CPU(s) on-line:    0-15
ID de fornecedor:             AuthenticAMD
  Nome do modelo:             AMD Ryzen 7 3700X 8-Core Processor
    Família da CPU:           23
    Modelo:                   113
    Thread(s) per núcleo:     2
    Núcleo(s) por soquete:    8
    Soquete(s):               1
    Step:                     0
    Frequency boost:          enabled
    CPU(s) scaling MHz:       72%
    CPU MHz máx.:             4426,1709
    CPU MHz mín.:             2200,0000
    BogoMIPS:                 7186,99
    Opções:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr p
                              ge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht
                               syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm cons
      

## 5. Verificação de Parâmetros de Armazenamento

### (a) Verificação de Parâmetros do Disco
    Modelo:ST1000DM010
    Informações obtidas de: https://www.seagate.com/www-content/product-content/barracuda-fam/barracuda-new/en-us/docs/100804187a.pdf
#### Número de Superfícies, Cilindros e Setores por Trilha
    16 superfícies,
    16.383 cilindros,
    63 setores por trilha.


#### Velocidade de Rotação e Latência Rotacional
    Rotação: 7200 RPM,
    Latência rotacional: 4,16 ms.


#### Tempos de Seek (Médio, Máximo e Mínimo)
    Tempo médio: 8,5 ms a 9,5 ms,
    Tempo máx: 12 ms,
    Tempo min: 0.5 ms a 1 ms.

#### Tempo para proxima trilha:
    Track-to-Track: 1.0 a 1.2 ms.

#### Taxa de Transferência
    Taxa de transferência: 600MB/s
            


### (b) Utilizando comando hdparm



In [10]:
!sudo hdparm /dev/sdb3



/dev/sdb3:
 multcount     = 16 (on)
 IO_support    =  1 (32-bit)
 readonly      =  0 (off)
 readahead     = 256 (on)
 geometry      = 121517/255/63, sectors = 1952173729, start = 239616


### (c) Tamanho do bloco e como alterá-lo:

#### Tamanho do bloco:


In [11]:
!sudo blockdev --getbsz /dev/sdb

4096


#### Alterando bloco:
    Rodando o comando: sudo mkfs.ext4 -b 4096 /dev/sdb
    Alteramos o tamanho do bloco para 4096.

In [12]:
!sudo mkfs.ext4 -b 4096 /dev/sdb3

mke2fs 1.47.0 (5-Feb-2023)
/dev/sdb3 está montado; não fará um sistema de arquivos aqui!


# Parte II

## 6. Analise e Descrição do armazenamento físico no PostgreSQL
- O PostgreSQL organiza seus arquivos em diretórios específicos, como base, global, pg_xlog, entre outros. Cada banco de dados possui seu próprio diretório dentro do diretório base.

- PostgreSQL usa a técnica TOAST (The Oversized-Attribute Storage Technique) para armazenar valores grandes que não cabem em uma única página de banco de dados. Esses valores são segmentados e armazenados em outra área secundária1.

- Os dados são armazenados em páginas de tamanho fixo (geralmente 8KB). Cada página contém uma estrutura de cabeçalho que descreve o conteúdo da página.

- Os índices são armazenados em páginas separadas e são usados para acelerar as consultas. Existem diferentes tipos de índices, como B-tree, Hash, GIN, entre outros.

- PostgreSQL utiliza várias técnicas de armazenamento, como compressão de dados, armazenamento em disco e memória compartilhada, para otimizar o desempenho e o uso de espaço.


### Tabela Comparativa

| Característica                | Descrição                                                                                     | Tamanho Padrão         |
|-------------------------------|-----------------------------------------------------------------------------------------------|-------------------------|
| **Bloco de Página**           | Unidade básica de armazenamento e I/O. Cada bloco possui cabeçalho e dados.                  | 8 KB (configurável)    |
| **Arquivos de Tabelas e Índices** | Arquivos físicos no sistema, divididos em segmentos menores quando > 1 GB.                  | Segmentos de até 1 GB   |
| **MVCC e Metadados de Transação** | Armazena múltiplas versões de linhas para controle de concorrência sem bloqueios.            | -                       |
| **TOAST**                     | Armazenamento externo para dados grandes que não cabem em uma página (ex: TEXT, BYTEA).      | -                       |
| **Log WAL**                   | Registro de todas as operações de modificação de dados para recuperação de falhas.           | 16 MB (padrão, ajustável) |
| **Tabelas de Catálogo**       | Meta-informações sobre tabelas, índices, usuários, permissões e configurações.               | -                       |
| **Armazenamento de Índices**  | Estruturas adicionais para otimizar consultas, com suporte para B-tree, Hash, GiST, etc.     | Depende do tipo de índice |


## Diagrama


## Análise dos Sistemas de Arquivo Disponíveis no Linux

### Ext2 (Second Extended Filesystem)
    Data de Lançamento: Janeiro de 1993
    
    Desenvolvedor: Remy Card
    
    Características Principais:
    
    Sem Journaling: Ext2 não possui journaling, o que significa que não há registro das operações de gravação e leitura, resultando em maior desempenho em dispositivos de armazenamento flash.
    
    Tamanho Máximo do Volume: 2 TiB a 32 TiB (dependendo do tamanho do bloco).
    
    Tamanho Máximo do Arquivo: 16 GiB a 2 TiB.
    
    Permissões de Arquivo: Suporta permissões Unix e listas de controle de acesso POSIX (ACLs).
    
    Tempo de Recuperação: Em caso de falha, a recuperação de um sistema de arquivos Ext2 pode ser demorada, pois todas as verificações de integridade devem ser realizadas.

### Ext3 (Third Extended Filesystem)
    Data de Lançamento: Novembro de 2001
    
    Desenvolvedor: Stephen Tweedie
    
    Características Principais:
    
    Com Journaling: Ext3 adiciona suporte a journaling, o que melhora a integridade dos dados após falhas no sistema.
    
    Tamanho Máximo do Volume: 2 TiB a 32 TiB.
    
    Tamanho Máximo do Arquivo: 16 GiB a 2 TiB.
    
    Permissões de Arquivo: Suporta permissões Unix e listas de controle de acesso POSIX (ACLs).
    
    Modos de Journaling: Três modos de journaling (writeback, ordered, journal) oferecem diferentes níveis de segurança e desempenho.

### ReiserFS (Reiser Filesystem)
    Data de Lançamento: 2001
    
    Desenvolvedor: Namesys
    
    Características Principais:
    
    Com Journaling: Suporta journaling, otimizado para pequenos arquivos, ideal para sistemas de arquivos que lidam com muitos arquivos pequenos.
    
    Tamanho Máximo do Volume: 4 TiB a 16 TiB.
    
    Tamanho Máximo do Arquivo: 2 GiB a 8 TiB.
    
    Permissões de Arquivo: Suporta permissões Unix e listas de controle de acesso POSIX (ACLs).
    
    Alocação Dinâmica: Eficiência na alocação de espaço, permitindo uso efetivo do espaço livre.

### XFS (XFS Filesystem)
    Data de Lançamento: 1994
    
    Desenvolvedor: Silicon Graphics
    
    Características Principais:
    
    Com Journaling: Suporta journaling, otimizado para grandes volumes de dados.
    
    Tamanho Máximo do Volume: 8 Exabytes.
    
    Tamanho Máximo do Arquivo: 8 Exabytes.
    
    Permissões de Arquivo: Suporta permissões Unix e listas de controle de acesso POSIX (ACLs).
    
    Desempenho Alto: Projetado para alto desempenho em operações de I/O intensivo, excelente para ambientes de grande escala.

### Tabela Comparativa
| Característica       | Ext2                  | Ext3                  | ReiserFS            | XFS                  |
|----------------------|-----------------------|-----------------------|---------------------|----------------------|
| **Journaling**       | Não                   | Sim                   | Sim                 | Sim                  |
| **Tamanho Máximo do Volume** | 2 TiB a 32 TiB        | 2 TiB a 32 TiB        | 4 TiB a 16 TiB      | 8 Exabytes           |
| **Tamanho Máximo do Arquivo** | 16 GiB a 2 TiB        | 16 GiB a 2 TiB        | 2 GiB a 8 TiB       | 8 Exabytes           |
| **Permissões de Arquivo** | Unix, POSIX ACLs        | Unix, POSIX ACLs        | Unix, POSIX ACLs      | Unix, POSIX ACLs      |
| **Eficiência em Pequenos Arquivos** | Moderada             | Moderada             | Alta                | Moderada             |
| **Desempenho em Grandes Volumes** | Moderado             | Moderado             | Moderado            | Alto                 |
| **Tempo de Recuperação** | Alto (sem journaling) | Baixo (com journaling) | Baixo (com journaling) | Baixo (com journaling) |


# Parte III

## 9. Preparação de Tabela Exemplo
### Tabela criada com os comandos:


In [30]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
DROP TABLE IF EXISTS t;
CREATE TABLE t (k serial PRIMARY KEY, v integer);
EOF

DROP TABLE
CREATE TABLE


In [31]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
INSERT INTO t(v) SELECT trunc(random() * 10)  FROM generate_series(1,100000);
EOF

INSERT 0 100000


### Consultando os valores das 10 primeiras tuplas da tabela, ordenando por k.

In [32]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
SELECT * FROM t ORDER BY k LIMIT 10;
EOF

 k  | v 
----+---
  1 | 8
  2 | 2
  3 | 2
  4 | 0
  5 | 1
  6 | 1
  7 | 4
  8 | 2
  9 | 9
 10 | 0
(10 linhas)



## 10. Verificando quantidade de páginas criadas

In [33]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
SELECT relname, relpages, reltuples FROM pg_class WHERE relname='t';
EOF

 relname | relpages | reltuples 
---------+----------+-----------
 t       |      443 |    100000
(1 linha)



## 11. Verificando a quantidade de blocos usados



In [34]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\pset x on
SELECT pg_sleep(1);
SELECT * FROM pg_stats WHERE tablename='t';
SELECT pg_stat_reset();
\pset x off
EOF

A exibição expandida está ativada.
-[ RECORD 1 ]
pg_sleep | 

-[ RECORD 1 ]----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
schemaname             | public
tablename              | t
attname                | k
inherited              | f
null_frac              | 0
avg_width              | 4
n_distinct             | -1
most_common_vals       | 
most_common_freqs      | 
histogram_bounds       | {2,945,1933,2899,3955,4967,6002,7103,8084,9130,

## 12. Indice
### Criando indice para o atributo v e tempo de consulta (número padrão de tuplas)

In [35]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
CREATE INDEX idx_v ON t(v);
EOF

CREATE INDEX


In [36]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF


Tempo de execução está ativado.
                                                       QUERY PLAN                                                       
------------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=121.80..696.26 rows=10517 width=8) (actual time=0.287..2.285 rows=10187 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=443
   ->  Bitmap Index Scan on idx_v  (cost=0.00..119.17 rows=10517 width=0) (actual time=0.244..0.244 rows=10187 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.387 ms
 Execution Time: 2.579 ms
(7 linhas)

Tempo: 3,452 ms


### Recriando indice para o atributo v:

In [38]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
DROP INDEX IF EXISTS idx_v;
CREATE INDEX idx_v on t(v);
EOF

Tempo de execução está ativado.
DROP INDEX
Tempo: 1,513 ms
CREATE INDEX
Tempo: 44,777 ms


### Criando indice para o atributo v e tempo de consulta com 10000000 de tuplas
#### Recriando a tabela

In [39]:
!sudo -u postgres psql -c "DROP TABLE IF EXISTS t;"
!sudo -u postgres psql -c "CREATE TABLE t (k serial PRIMARY KEY, v integer);"

DROP TABLE
CREATE TABLE


In [40]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
INSERT INTO t(v) SELECT trunc(random() * 10)  FROM generate_series(1,10000000);
EOF

INSERT 0 10000000


In [41]:
!sudo -u postgres psql -c "CREATE INDEX idx_v ON t(v);"

CREATE INDEX


In [42]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                        QUERY PLAN                                                         
---------------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=559.93..47536.70 rows=50000 width=8) (actual time=28.507..309.351 rows=999223 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=44248
   ->  Bitmap Index Scan on idx_v  (cost=0.00..547.43 rows=50000 width=0) (actual time=22.955..22.956 rows=999223 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.270 ms
 Execution Time: 335.290 ms
(7 linhas)

Tempo: 336,172 ms


### Recriando indice para o atributo v com 10000000 tuplas:

In [43]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
DROP INDEX IF EXISTS idx_v;
CREATE INDEX idx_v on t(v);
EOF

Tempo de execução está ativado.
DROP INDEX
Tempo: 15,924 ms
CREATE INDEX
Tempo: 4111,294 ms (00:04,111)


## 13. Fill factor
### Retornando a tabela para o tamanho padrão.

In [44]:
!sudo -u postgres psql -c "DROP TABLE IF EXISTS t;"
!sudo -u postgres psql -c "CREATE TABLE t (k serial PRIMARY KEY, v integer);"

DROP TABLE
CREATE TABLE


In [45]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
INSERT INTO t(v) SELECT trunc(random() * 10)  FROM generate_series(1,100000);
EOF

INSERT 0 100000


### Criando indice para o atributo v e tempo de consulta (número padrão de tuplas):
### Fill factor 60 -> comando usado:

In [46]:
!sudo -u postgres psql -c "CREATE INDEX idx_v ON t(v);" -c "ALTER TABLE t SET (fillfactor = 60);" -c "VACUUM FULL t;"

CREATE INDEX
ALTER TABLE
VACUUM


In [47]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                      QUERY PLAN                                                      
----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=113.66..979.45 rows=9983 width=8) (actual time=0.332..2.107 rows=9991 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=741
   ->  Bitmap Index Scan on idx_v  (cost=0.00..111.17 rows=9983 width=0) (actual time=0.267..0.267 rows=9991 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.240 ms
 Execution Time: 2.386 ms
(7 linhas)

Tempo: 3,078 ms


### Fill factor 80 -> comando usado:

In [48]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 80);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [49]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                      QUERY PLAN                                                      
----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=113.66..791.45 rows=9983 width=8) (actual time=0.346..1.989 rows=9991 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=553
   ->  Bitmap Index Scan on idx_v  (cost=0.00..111.17 rows=9983 width=0) (actual time=0.294..0.294 rows=9991 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.348 ms
 Execution Time: 2.286 ms
(7 linhas)

Tempo: 3,057 ms


### Fill factor 90 -> comando usado:

In [50]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 90);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [51]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                      QUERY PLAN                                                      
----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=113.66..729.45 rows=9983 width=8) (actual time=0.366..2.225 rows=9991 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=491
   ->  Bitmap Index Scan on idx_v  (cost=0.00..111.17 rows=9983 width=0) (actual time=0.309..0.310 rows=9991 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.332 ms
 Execution Time: 2.518 ms
(7 linhas)

Tempo: 3,308 ms


### Fill factor 100 -> comando usado:

In [52]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 100);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [53]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                      QUERY PLAN                                                      
----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=113.66..681.45 rows=9983 width=8) (actual time=0.280..2.279 rows=9991 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=443
   ->  Bitmap Index Scan on idx_v  (cost=0.00..111.17 rows=9983 width=0) (actual time=0.239..0.239 rows=9991 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.294 ms
 Execution Time: 2.566 ms
(7 linhas)

Tempo: 3,329 ms


### Recriando a tabela para 1000000 tuplas

In [54]:
!sudo -u postgres psql -c "DROP TABLE IF EXISTS t;"
!sudo -u postgres psql -c "CREATE TABLE t (k serial PRIMARY KEY, v integer);"

DROP TABLE
CREATE TABLE


In [55]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
INSERT INTO t(v) SELECT trunc(random() * 10)  FROM generate_series(1,10000000);
EOF

INSERT 0 10000000


### Criando indice para o atributo v e tempo de consulta com 10000000 de tuplas
### Fill factor 60 -> comando usado:

In [56]:
!sudo -u postgres psql -c "CREATE INDEX idx_v ON t(v);" -c "ALTER TABLE t SET (fillfactor = 60);" -c "VACUUM FULL t;"

CREATE INDEX
ALTER TABLE
VACUUM


In [57]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                        QUERY PLAN                                                         
---------------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=559.93..71047.40 rows=50000 width=8) (actual time=36.982..633.690 rows=999949 loops=1)
   Recheck Cond: (v = 5)
   Rows Removed by Index Recheck: 4012621
   Heap Blocks: exact=41050 lossy=33025
   ->  Bitmap Index Scan on idx_v  (cost=0.00..547.43 rows=50000 width=0) (actual time=31.213..31.214 rows=999949 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.299 ms
 Execution Time: 660.479 ms
(8 linhas)

Tempo: 661,409 ms


### Fill factor 80 -> comando usado:

In [58]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 80);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [59]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                        QUERY PLAN                                                         
---------------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=559.93..57363.40 rows=50000 width=8) (actual time=35.353..326.741 rows=999949 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=55249
   ->  Bitmap Index Scan on idx_v  (cost=0.00..547.43 rows=50000 width=0) (actual time=28.074..28.074 rows=999949 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.313 ms
 Execution Time: 353.345 ms
(7 linhas)

Tempo: 354,333 ms


### Fill factor 90 -> comando usado:

In [60]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 90);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [61]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                          QUERY PLAN                                                           
-------------------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=11470.90..73365.83 rows=1029995 width=8) (actual time=33.456..300.521 rows=999949 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=49020
   ->  Bitmap Index Scan on idx_v  (cost=0.00..11213.40 rows=1029995 width=0) (actual time=27.142..27.143 rows=999949 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.253 ms
 Execution Time: 326.529 ms
(7 linhas)

Tempo: 327,354 ms


### Fill factor 100 -> comando usado:

In [62]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 100);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [63]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                          QUERY PLAN                                                           
-------------------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=11470.93..68593.93 rows=1030000 width=8) (actual time=29.130..282.451 rows=999949 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=44248
   ->  Bitmap Index Scan on idx_v  (cost=0.00..11213.43 rows=1030000 width=0) (actual time=23.326..23.327 rows=999949 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.325 ms
 Execution Time: 308.480 ms
(7 linhas)

Tempo: 309,385 ms


## 15. Indices Decrescentes
## - Recriando tabela com o tamanho de tuplas padrão

In [64]:
!sudo -u postgres psql -c "DROP TABLE IF EXISTS t;"
!sudo -u postgres psql -c "CREATE TABLE t (k serial PRIMARY KEY, v integer);"

DROP TABLE
CREATE TABLE


In [66]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
INSERT INTO t(v) SELECT trunc(random() * 10)  FROM generate_series(1,100000);
EOF

INSERT 0 100000


### Criando indice para o atributo v e tempo de consulta (número padrão de tuplas):
### Fill factor 60 -> comando usado:

In [67]:
!sudo -u postgres psql -c "CREATE INDEX i ON t(v DESC NULLS FIRST);"

CREATE INDEX


In [68]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 60);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [69]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                     QUERY PLAN                                                     
--------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=226.83..1958.08 rows=19940 width=8) (actual time=0.634..4.515 rows=19882 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=1482
   ->  Bitmap Index Scan on i  (cost=0.00..221.84 rows=19940 width=0) (actual time=0.499..0.499 rows=19882 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.357 ms
 Execution Time: 5.089 ms
(7 linhas)

Tempo: 6,410 ms


### Fill factor 80 -> comando usado:

In [70]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 80);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [72]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                     QUERY PLAN                                                     
--------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=226.83..1581.08 rows=19940 width=8) (actual time=0.549..3.816 rows=19882 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=1105
   ->  Bitmap Index Scan on i  (cost=0.00..221.84 rows=19940 width=0) (actual time=0.452..0.452 rows=19882 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.285 ms
 Execution Time: 4.391 ms
(7 linhas)

Tempo: 5,165 ms


### Fill factor 90 -> comando usado:

In [73]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 90);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [74]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                     QUERY PLAN                                                     
--------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=226.83..1457.08 rows=19940 width=8) (actual time=0.607..4.255 rows=19882 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=981
   ->  Bitmap Index Scan on i  (cost=0.00..221.84 rows=19940 width=0) (actual time=0.508..0.508 rows=19882 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.300 ms
 Execution Time: 4.828 ms
(7 linhas)

Tempo: 5,598 ms


### Fill factor 100 -> comando usado:

In [75]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 100);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [76]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                     QUERY PLAN                                                     
--------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=226.83..1361.08 rows=19940 width=8) (actual time=0.515..4.261 rows=19882 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=885
   ->  Bitmap Index Scan on i  (cost=0.00..221.84 rows=19940 width=0) (actual time=0.441..0.441 rows=19882 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.306 ms
 Execution Time: 4.835 ms
(7 linhas)

Tempo: 5,654 ms


### Recriando indice para o atributo v:

In [77]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
DROP INDEX IF EXISTS i;
CREATE INDEX i ON t(v DESC NULLS FIRST);
EOF

Tempo de execução está ativado.
DROP INDEX
Tempo: 1,578 ms
CREATE INDEX
Tempo: 92,168 ms


## - Recriando a tabela para 1000000 tuplas

In [78]:
!sudo -u postgres psql -c "DROP TABLE IF EXISTS t;"
!sudo -u postgres psql -c "CREATE TABLE t (k serial PRIMARY KEY, v integer);"

DROP TABLE
CREATE TABLE


In [79]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
INSERT INTO t(v) SELECT trunc(random() * 10)  FROM generate_series(1,10000000);
EOF

INSERT 0 10000000


### Fill factor 60 -> comando usado:

In [80]:
!sudo -u postgres psql -c "CREATE INDEX i ON t(v DESC NULLS FIRST);"

CREATE INDEX


In [81]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 60);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [82]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                      QUERY PLAN                                                       
-----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=559.93..71047.40 rows=50000 width=8) (actual time=33.212..638.288 rows=999612 loops=1)
   Recheck Cond: (v = 5)
   Rows Removed by Index Recheck: 4013045
   Heap Blocks: exact=41050 lossy=33025
   ->  Bitmap Index Scan on i  (cost=0.00..547.43 rows=50000 width=0) (actual time=27.985..27.986 rows=999612 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.285 ms
 Execution Time: 665.142 ms
(8 linhas)

Tempo: 666,039 ms


### Fill factor 80 -> comando usado:

In [83]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 80);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [84]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                      QUERY PLAN                                                       
-----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=559.93..57363.40 rows=50000 width=8) (actual time=34.403..322.909 rows=999612 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=55249
   ->  Bitmap Index Scan on i  (cost=0.00..547.43 rows=50000 width=0) (actual time=27.057..27.058 rows=999612 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.342 ms
 Execution Time: 349.375 ms
(7 linhas)

Tempo: 350,355 ms


### Fill factor 90 -> comando usado:

In [85]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 90);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [86]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                      QUERY PLAN                                                       
-----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=559.93..51991.41 rows=50000 width=8) (actual time=31.515..309.945 rows=999612 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=49020
   ->  Bitmap Index Scan on i  (cost=0.00..547.43 rows=50000 width=0) (actual time=25.143..25.144 rows=999612 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.301 ms
 Execution Time: 336.662 ms
(7 linhas)

Tempo: 337,581 ms


### Fill factor 100 -> comando usado:

In [87]:
!sudo -u postgres psql -c "ALTER TABLE t SET (fillfactor = 100);" -c "VACUUM FULL t;"

ALTER TABLE
VACUUM


In [88]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
EXPLAIN ANALYZE SELECT * FROM t WHERE v = 5;
EOF

Tempo de execução está ativado.
                                                      QUERY PLAN                                                       
-----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on t  (cost=559.93..47536.70 rows=50000 width=8) (actual time=29.509..282.118 rows=999612 loops=1)
   Recheck Cond: (v = 5)
   Heap Blocks: exact=44248
   ->  Bitmap Index Scan on i  (cost=0.00..547.43 rows=50000 width=0) (actual time=23.928..23.928 rows=999612 loops=1)
         Index Cond: (v = 5)
 Planning Time: 0.324 ms
 Execution Time: 308.107 ms
(7 linhas)

Tempo: 309,070 ms


### Recriando indice para o atributo v:

In [89]:
%%bash
sudo -u postgres psql -v ON_ERROR_STOP=1 <<EOF
\timing
DROP INDEX IF EXISTS i;
CREATE INDEX i ON t(v DESC NULLS FIRST);
EOF

Tempo de execução está ativado.
DROP INDEX
Tempo: 17,467 ms
CREATE INDEX
Tempo: 5112,644 ms (00:05,113)


## 16. Preparação e Verificação do Ambiente
### a) Nome do índice e nome da tabela

In [91]:
!sudo -u postgres createdb tp3

In [ ]:
!sudo -u postgres psql -d tp3 -f movie.sql

DROP TABLE
CREATE TABLE
CREATE INDEX
CREATE INDEX
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSE

In [104]:
!sudo -u postgres psql -d tp3 -f actor.sql

psql:actor.sql:1: ERRO:  a tabela "actor" não existe
psql:actor.sql:3: AVISO:  não há uma transação em execução
COMMIT
CREATE TABLE
CREATE INDEX
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0

In [105]:
!sudo -u postgres psql -d tp3 -f casting.sql

psql:casting.sql:1: ERRO:  a tabela "casting" não existe
psql:casting.sql:3: AVISO:  não há uma transação em execução
COMMIT
CREATE TABLE
CREATE INDEX
CREATE INDEX
CREATE INDEX
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 

### b) Altura do índice, número de chaves por bloco e outras informações detalhadas:

In [106]:

!sudo -u postgres psql -d tp3 -P pager=off -c "SELECT \
    c.relname AS index_name, \
    t.relname AS table_name, \
    c.reltuples AS num_chaves, \
    c.relpages AS num_blocos, \
    i.indisunique AS is_unique, \
    i.indisprimary AS is_primary, \
    pg_catalog.pg_size_pretty(pg_relation_size(c.oid)) AS index_size, \
    c.reltuples / NULLIF(c.relpages, 0) AS chaves_por_bloco \
FROM \
    pg_class c \
JOIN \
    pg_index i ON c.oid = i.indexrelid \
JOIN \
    pg_class t ON i.indrelid = t.oid \
WHERE \
    t.relname IN ('movie', 'actor', 'casting');"



  index_name   | table_name | num_chaves | num_blocos | is_unique | is_primary | index_size |  chaves_por_bloco  
---------------+------------+------------+------------+-----------+------------+------------+--------------------
 actor_id      | actor      |       5944 |         19 | f         | f          | 152 kB     |  312.8421052631579
 movie_key     | movie      |       1844 |          7 | t         | t          | 56 kB      | 263.42857142857144
 movie_title   | movie      |       1844 |         12 | f         | f          | 96 kB      | 153.66666666666666
 movie_votes   | movie      |       1844 |         11 | f         | f          | 88 kB      | 167.63636363636363
 casting_movie | casting    |          0 |          1 | f         | f          | 216 kB     |                  0
 casting_actor | casting    |          0 |          1 | f         | f          | 264 kB     |                  0
 casting_ord   | casting    |          0 |          1 | f         | f          | 160 kB     | 

## 17. Consultas por intervalo e índices secundários
### a) EXPLAIN sobre uma consulta de um numero pequeno de tuplas (<10 tuplas).

In [107]:
!sudo -u postgres psql -d tp3 -P pager=off -c "EXPLAIN ANALYZE\
SELECT * FROM movie \
WHERE votes < 50 \
ORDER BY votes \
LIMIT 9;"

                                                        QUERY PLAN                                                        
--------------------------------------------------------------------------------------------------------------------------
 Limit  (cost=0.28..4.30 rows=1 width=30) (actual time=0.007..0.007 rows=0 loops=1)
   ->  Index Scan using movie_votes on movie  (cost=0.28..4.30 rows=1 width=30) (actual time=0.006..0.006 rows=0 loops=1)
         Index Cond: (votes < 50)
 Planning Time: 0.552 ms
 Execution Time: 0.042 ms
(5 linhas)



### b) Explain sobre uma consulta de um numero grande de tuplas (>80% das tuplas)

In [108]:
!sudo -u postgres psql -d tp3 -P pager=off -c "EXPLAIN ANALYZE\
SELECT * FROM movie \
WHERE votes > 1000;"

                                              QUERY PLAN                                               
-------------------------------------------------------------------------------------------------------
 Seq Scan on movie  (cost=0.00..38.05 rows=1515 width=30) (actual time=0.009..0.173 rows=1516 loops=1)
   Filter: (votes > 1000)
   Rows Removed by Filter: 328
 Planning Time: 0.392 ms
 Execution Time: 0.254 ms
(5 linhas)



### c) Explique porque o índice sobre VOTES não é sempre usado nas consultas sobre este atributo. 
O PostgreSQL utiliza o índice sobre votes somente quando a consulta é seletiva o suficiente (geralmente quando um número pequeno de tuplas é retornado). Quando a condição é muito ampla (como no caso de votes > 1000), o Seq Scan se torna mais eficiente, pois evitaria o custo de acessar o índice e realizar múltiplas buscas. Em casos de grandes volumes de dados, a varredura sequencial da tabela é preferível por ser mais rápida.

## 18. Comparações de Operadores de Agregação
### a) Apresente o resultado do comando explain sobre as duas consultas acima

In [116]:
!sudo -u postgres psql -d tp3 -P pager=off -c "EXPLAIN ANALYZE \
SELECT title \
FROM movie \
WHERE votes >= (SELECT MAX(votes) FROM movie);"

                                                                            QUERY PLAN                                                                             
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Index Scan using movie_votes on movie  (cost=0.61..35.37 rows=615 width=16) (actual time=0.035..0.036 rows=1 loops=1)
   Index Cond: (votes >= (InitPlan 2).col1)
   InitPlan 2
     ->  Result  (cost=0.32..0.33 rows=1 width=4) (actual time=0.025..0.026 rows=1 loops=1)
           InitPlan 1
             ->  Limit  (cost=0.28..0.32 rows=1 width=4) (actual time=0.024..0.024 rows=1 loops=1)
                   ->  Index Only Scan Backward using movie_votes on movie movie_1  (cost=0.28..76.55 rows=1844 width=4) (actual time=0.024..0.024 rows=1 loops=1)
                         Index Cond: (votes IS NOT NULL)
                         Heap Fetches: 0
 Planning Time: 0.540

In [117]:
!sudo -u postgres psql -d tp3 -P pager=off -c "EXPLAIN ANALYZE \
SELECT title \
FROM movie \
WHERE votes >= ALL (SELECT votes FROM movie);"

                                                         QUERY PLAN                                                         
----------------------------------------------------------------------------------------------------------------------------
 Seq Scan on movie  (cost=0.00..43620.99 rows=922 width=16) (actual time=0.509..0.779 rows=1 loops=1)
   Filter: (ALL (votes >= (SubPlan 1).col1))
   Rows Removed by Filter: 1843
   SubPlan 1
     ->  Materialize  (cost=0.00..42.66 rows=1844 width=4) (actual time=0.000..0.000 rows=2 loops=1844)
           ->  Seq Scan on movie movie_1  (cost=0.00..33.44 rows=1844 width=4) (actual time=0.001..0.175 rows=1844 loops=1)
 Planning Time: 0.360 ms
 Execution Time: 0.834 ms
(8 linhas)



### b) Existe alguma diferença entre os planos de consultas? Qual das duas é mais eficiente? Explique?

A primeira consulta usa um **Index Scan**, que é mais eficiente porque consulta diretamente o índice para determinar o valor máximo de votes.
A segunda consulta realiza um **Seq Scan** na tabela movie, que é menos eficiente, especialmente quando a tabela tem muitas tuplas. 

**A primeira consulta é mais eficiente**. O uso de um Index Scan para buscar o valor máximo de votes e filtrar os resultados é muito mais rápido e econômico em termos de tempo de execução.

## 19. Consultas com Junção e Seleção
### a) rodando consulta


In [111]:
!sudo -u postgres psql -d tp3 -P pager=off -c "EXPLAIN ANALYZE SELECT title \
FROM movie \
WHERE votes > (SELECT votes FROM movie WHERE title = 'Star Wars');"


                                                            QUERY PLAN                                                             
-----------------------------------------------------------------------------------------------------------------------------------
 Index Scan using movie_votes on movie  (cost=8.57..43.34 rows=615 width=16) (actual time=0.036..0.036 rows=0 loops=1)
   Index Cond: (votes > (InitPlan 1).col1)
   InitPlan 1
     ->  Index Scan using movie_title on movie movie_1  (cost=0.28..8.29 rows=1 width=4) (actual time=0.024..0.024 rows=1 loops=1)
           Index Cond: ((title)::text = 'Star Wars'::text)
 Planning Time: 0.431 ms
 Execution Time: 0.079 ms
(7 linhas)



In [114]:
!sudo -u postgres psql -d tp3 -P pager=off -c "EXPLAIN ANALYZE SELECT m1.title \
FROM movie m1, movie m2 \
WHERE m1.votes > m2.votes AND m2.title = 'Star Wars';"


                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.56..49.49 rows=615 width=16) (actual time=0.029..0.030 rows=0 loops=1)
   ->  Index Scan using movie_title on movie m2  (cost=0.28..8.29 rows=1 width=4) (actual time=0.019..0.020 rows=1 loops=1)
         Index Cond: ((title)::text = 'Star Wars'::text)
   ->  Index Scan using movie_votes on movie m1  (cost=0.28..35.04 rows=615 width=20) (actual time=0.008..0.008 rows=0 loops=1)
         Index Cond: (votes > m2.votes)
 Planning Time: 0.519 ms
 Execution Time: 0.073 ms
(7 linhas)



### b) Existe alguma diferença entre os planos de consultas? Qual das duas é mais eficiente? Explique?

A primeira consulta utiliza um **Index Scan** na tabela movie para buscar os votos de "Star Wars", depois faz um Index Scan na tabela para buscar os filmes com votos maiores que esse valor.

A segunda realiza um **Nested Loop Join**. A consulta interna busca "Star Wars" através de um Index Scan, e então compara os votos de todos os filmes (m1) com os votos de "Star Wars" (m2).

A **primeira é mais eficiente** porque realiza a consulta com uma subconsulta escalar que retorna um único valor (votos de "Star Wars"). Ele só precisa buscar esse valor uma vez, e depois filtra os filmes.

A segunda consulta (junção implícita) pode ser menos eficiente devido ao Nested Loop Join, o que pode levar a mais comparações e maior custo, dependendo do tamanho da tabela.

## 20. Casamento de Strings e Índices

### a) Resultado do comando EXPLAIN sobre as consultas:

In [118]:
!sudo -u postgres psql -d tp3 -P pager=off -c "EXPLAIN SELECT title \
FROM movie \
WHERE title LIKE 'I%';"


                       QUERY PLAN                       
--------------------------------------------------------
 Seq Scan on movie  (cost=0.00..38.05 rows=18 width=16)
   Filter: ((title)::text ~~ 'I%'::text)
(2 linhas)



In [119]:
!sudo -u postgres psql -d tp3 -P pager=off -c "EXPLAIN SELECT title \
FROM movie \
WHERE substr(title, 1, 1) = 'I';"


                      QUERY PLAN                       
-------------------------------------------------------
 Seq Scan on movie  (cost=0.00..42.66 rows=9 width=16)
   Filter: (substr((title)::text, 1, 1) = 'I'::text)
(2 linhas)



In [120]:
!sudo -u postgres psql -d tp3 -P pager=off -c "EXPLAIN SELECT title \
FROM movie \
WHERE title LIKE '%A';"

                       QUERY PLAN                       
--------------------------------------------------------
 Seq Scan on movie  (cost=0.00..38.05 rows=18 width=16)
   Filter: ((title)::text ~~ '%A'::text)
(2 linhas)



Em todas essas consultas, o PostgreSQL escolhe realizar um Seq Scan porque os operadores LIKE e a função substr() não são adequados para uso eficiente de índices quando são aplicados dessa maneira. 

### b) Qual das três apresenta o menor custo? Porque? 
A consulta **LIKE 'I%'** e a consulta **LIKE '%A'** têm o menor custo, com um custo de 38.05. A consulta substr(title, 1, 1) = 'I' tem o maior custo (42.66) devido ao uso de uma função, que impede o uso de índices e resulta em mais processamento.

### c) O índice sobre TITLE foi usado para todas elas? Justifique. 

O índice sobre TITLE não foi utilizado nas três consultas porque:

Na consulta LIKE 'I%', o PostgreSQL optou por uma varredura sequencial, provavelmente por considerar que o índice não traria um benefício significativo.

Na consulta substr(title, 1, 1) = 'I', a função substr() impede o uso de índices, já que ele não pode ser aplicado diretamente sobre o valor do campo.

Na consulta LIKE '%A', o % no início do padrão de busca também impede o uso do índice, já que esse tipo de consulta não pode ser otimizado por um índice B-tree.

## 21. Verificação da hipótese de distribuição uniforme na estimativa de seletividade

### a) Apresente o resultado do comando explain sobre as duas consultas acima. Explique o resultado.

In [121]:
!sudo -u postgres psql -d tp3 -P pager=off -c "EXPLAIN SELECT title FROM movie WHERE votes < 1000;"

                                 QUERY PLAN                                  
-----------------------------------------------------------------------------
 Index Scan using movie_votes on movie  (cost=0.28..20.02 rows=328 width=16)
   Index Cond: (votes < 1000)
(2 linhas)



In [122]:
!sudo -u postgres psql -d tp3 -P pager=off -c "EXPLAIN SELECT title FROM movie WHERE votes > 40000;"

                                QUERY PLAN                                
--------------------------------------------------------------------------
 Index Scan using movie_votes on movie  (cost=0.28..8.42 rows=8 width=16)
   Index Cond: (votes > 40000)
(2 linhas)



Na primeira consulta PostgreSQL usou um índice (provavelmente um índice B-tree) sobre o campo votes para realizar uma varredura do índice (Index Scan). A condição votes < 1000 foi aplicada diretamente sobre o índice, o que torna essa consulta mais eficiente do que uma varredura sequencial.

Na segunda consulta um índice sobre o campo votes foi utilizado, mas desta vez para a condição votes > 40000. Isso resultou em um número muito menor de tuplas (8 tuplas) e, portanto, o custo da operação foi mais baixo (0.28 a 8.42).

### b) Compare o número de tuplas selecionadas por cada consulta. Qual das duas tem a menor seletividade?

A consulta com a menor seletividade é a Consulta 2, pois ela retorna um número muito menor de tuplas em comparação com o total de registros no banco. 

# PARTE V

Questões 22, 23 e 24 são feitas diretamente na execução do aruquivo "parte5.py"

## Aviso para questão 24
 - A questão 24 pede uma tabela com o número de vezes que uma reserva teve que ser refeita. 
 para isso o código foi feito de maneira em que propositalmente ocorram tentativas para tentar reservar, isso pode ser evitado 